Code to test the effectiveness of term similarity through Cosine Similarity metrics

In [184]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import cleaner

Loading global tagger... please wait a few seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [208]:
df_prods = pd.read_csv('../data/clean_product_descriptions.csv').head(n=100)


In [224]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords


vect = CountVectorizer(min_df=1, stop_words=stopwords.words('english'), ngram_range=(1,2))
X_vect = vect.fit_transform(df_prods['product_description_cleaned'])

In [205]:
tfidf = TfidfVectorizer(min_df=1, stop_words=stopwords.words('english'))
X_tfidf = tfidf.fit_transform(df_prods['product_description_cleaned'])

In [206]:
# How to convert a sentance to a vector
# gives back a 1 x p vector and counts the occurences
print vect.transform(['common angles'])

  (0, 21260)	1
  (0, 56150)	1


# Comapre with the search term

In [215]:
df_train = pd.read_csv('../data/train_sample.csv')

In [257]:
vect = TfidfVectorizer(min_df=1, stop_words=stopwords.words('english'), ngram_range=(1,2), )
vect.fit_transform([
        'hello there',
        'fat cat',
        'hello cats are skinny'
    ])
vect.transform(['hello there hello']).A

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]])

In [258]:
vect.get_feature_names()

[u'cat',
 u'cats',
 u'cats skinny',
 u'fat',
 u'fat cat',
 u'hello',
 u'hello cats',
 u'skinny']

In [217]:
from sklearn.metrics.pairwise import cosine_similarity

In [231]:
cosine_similarity(X_vect[8], vect.transform([df_train.loc[0]['search_term']]))

array([[ 0.]])

In [238]:
vect.fit_transform([
        'hello there',
        'cats are fat'
    ])

<2x4 sparse matrix of type '<type 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [241]:
vect.get_feature_names()

[u'cats', u'cats fat', u'fat', u'hello']

In [236]:
vect.transform(['hello there'])

<1x4 sparse matrix of type '<type 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [218]:

row_num = 2
row = df_train.loc[row_num]
print row.relevance
print row['search_term']
print row['product_description']

3.0
exterior door hinges
The Sea Gull Lighting Montclaire 2-light vanity fixture in olde iron is an Energy Star qualified lighting fixture that uses fluorescent bulbs to save you both time and money. The Montclaire collection from Sea Gull Lighting brings together subtle European flair with soft lines to create a transitional style for today's homes. When selected in Olde Iron, the deep, rich finish, combined with amber scavo glass gives the Montclaire collection a taste of rustic charm. Choose from an entire collection of indoor lighting fixtures to light the entirety of your home with Montclaire, from Sea Gull Lighting.Featured in the decorative Montclaire collectionUses 2 self-ballasted PLS13 GU24 13-Watt light bulbsHighlighted with etched white alabaster glass shadeHighlighted with amber scavo glass shadeEnergy Star qualifiedOffers energy saving fluorescent lightingFixture may be mounted in the up or down positionSafety listed for use in damp locationsGreat choice for your do-it-yo

In [220]:
def get_cosine_similarity(row, df_corpus, vectorizer, X):
    row_num = df_corpus[df_corpus['product_uid'] == row['product_uid']].index
    return cosine_similarity(X[row_num],vectorizer.transform([row['search_term']])).tolist()[0][0]
df_train['cosine_count'] = df_train.apply(get_cosine_similarity, axis=1, args = (df_prods, vect, X_vect))
# df_train['cosine_tfidf'] = df_train.apply(get_cosine_similarity, axis=1, args = (df_prods, tfidf, X_tfidf))

ValueError: ('Found array with 0 sample(s) (shape=(0, 3140)) while a minimum of 1 is required.', u'occurred at index 1')

In [ ]:
df_train[['cosine_count', 'cosine_tfidf']]

In [ ]:
plt.scatter(df_train['cosine_tfidf'], df_train['relevance'] )

In [ ]:
ax = plt.subplot(111, projection='3d')
ax.scatter(df_train['cosine_tfidf'], df_train['cosine_count'],
               zs=df_train['relevance'], s=20, c='b', depthshade=True)

In [ ]:

row_num = df_prods[df_prods['product_uid'] == row['product_uid']].index



In [ ]:

row_num = df_prods[df_prods['product_uid'] == row['product_uid']].index
cosine_similarity(X_tfidf[row_num],tfidf.transform([row['search_term']]))

In [105]:
vect_singular = CountVectorizer(min_df=1, stop_words=stopwords.words('english'))
X_sing = vect_singular.fit_transform(row[['product_description', 'search_term']])
cosine_similarity(X_sing[0],X_sing[1])

array([[ 0.]])